None {1: None, 'Browse': None, 3: None, 4: None, 5: None, 6: None, 7: None}


In [108]:
import PySimpleGUI as sg

sg.theme("lightGrey2")  # No gray windows please!
# 컴포넌트 정의
# STEP 1 define the layout


left_column = [
    [sg.Image(r"C:\Users\jundo\Graduation Project\sample_img.png", size=(200, 200))],
    [sg.Text("리셉터 파일을 업로드" , size=(30, 1),justification='center')],
    [sg.Input("파일 경로 입력...", size=(25, 1)), sg.FileBrowse(button_text='찾아보기...')],
    [sg.Text("_" * 35)],
    [sg.Text("추천되는 파라미터" , size=(30, 1),justification='center')],
    [sg.Text("exhaustiveness : 8 \n Grid 8 8 8 ",size=(30, 3),justification='center')],
]


right_column = [
    [sg.Text("col Row 1")],
    [
        sg.Frame(
            "My Frame Title",
            [
                [sg.T("리간드 파일 선택")],
                [
                    sg.Column(
                        [
                            [sg.Image(r"folder_icon.png", size=(60, 60))],
                            [sg.Text("리셉터 폴더 선택")],
                        ],
                        element_justification="center",
                    ),
                    sg.Column(
                        [
                            [sg.Image(r"db_icon.png", size=(60, 60))],
                            [sg.Text("온라인 DB 사용")],
                        ],
                        element_justification="center",
                    ),
                ],
                [sg.Input("폴더 경로 입력...", size=(25, 1)), sg.FolderBrowse(button_text='찾아보기...')],
            ],
        )
    ],
    [
        sg.Table(
            [["Dapagliflozin (1)", "대기중", "--:--:--", "--", "--"], [4, 5, 6, 7, 8]],
            ["리간드 이름", "Docking 수행", "Docking 수행 시간", "결과 평가", "사용한 파라미터"],
            num_rows=2,
            justification="center",
        )
    ],
]


layout = [
    [sg.Column(left_column, expand_y = True), sg.VerticalSeparator(pad=None), sg.Column(right_column,expand_y = True)],
    [sg.Button("Button"), sg.Button("Exit")],
]

column = [[sg.Column(layout)]]
# STEP 2 - create the window
window = sg.Window("My new window", column, grab_anywhere=True)

# STEP3 - the event loop
while True:
    (
        event,
        values,
    ) = window.read()  # Read the event that happened and the values dictionary
    print(event, values)
    if event in (
        None,
        "Exit",
    ):  # If user closeddow with X or if user clicked "Exit" button then exit
        break
    if event == "Button":
        print("You pressed the button")
window.close()


KeyboardInterrupt: 